<a href="https://colab.research.google.com/github/olid-iu/OH110032/blob/main/CNN_Patato_Leaf_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [5]:
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mdolidhasan","key":"c664b40086891d359002d3db42a35203"}'}

In [6]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [7]:

!kaggle datasets download -d warcoder/potato-leaf-disease-dataset


Dataset URL: https://www.kaggle.com/datasets/warcoder/potato-leaf-disease-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 98% 705M/719M [00:08<00:00, 131MB/s]
100% 719M/719M [00:08<00:00, 91.2MB/s]


In [8]:
import zipfile
import os

# Unzip the dataset
with zipfile.ZipFile("potato-leaf-disease-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("potato_leaf_data")

In [9]:
import os

# Check the extracted files
base_dir = '/content/potato_leaf_data/Potato Leaf Disease Dataset in Uncontrolled Environment'
print(os.listdir(base_dir))


['Fungi', 'Healthy', 'Bacteria', 'Virus', 'Nematode', 'Pest', 'Phytopthora']


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set parameters
img_height, img_width = 128, 128  # Resize images
batch_size = 32

# Use ImageDataGenerator for data augmentation and loading
datagen = ImageDataGenerator(
    rescale=1.0/255,                # Normalize pixel values
    validation_split=0.2,            # Split for training and validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Training data generator
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)


Found 2464 images belonging to 7 classes.
Found 612 images belonging to 7 classes.


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define a simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 4, 4, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 544,007 (2.08 MB)

 Trainable params: 544,007 (2.08 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Train the model
epochs = 30  # You can increase this number for better results

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    verbose=1
)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


77/77 ━━━━━━━━━━━━━━━━━━━━ 55s 580ms/step - accuracy: 0.2240 - loss: 1.8189 - val_accuracy: 0.2484 - val_loss: 1.7984
Epoch 2/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 45s 549ms/step - accuracy: 0.2675 - loss: 1.7268 - val_accuracy: 0.3154 - val_loss: 1.8624
Epoch 3/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 82s 545ms/step - accuracy: 0.3586 - loss: 1.5817 - val_accuracy: 0.2304 - val_loss: 2.0007
Epoch 4/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 46s 559ms/step - accuracy: 0.3495 - loss: 1.5581 - val_accuracy: 0.3088 - val_loss: 1.8622
Epoch 5/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 80s 540ms/step - accuracy: 0.3748 - loss: 1.4794 - val_accuracy: 0.3235 - val_loss: 1.7756
Epoch 6/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 45s 536ms/step - accuracy: 0.3913 - loss: 1.4673 - val_accuracy: 0.3497 - val_loss: 1.8551
Epoch 7/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 80s 525ms/step - accuracy: 0.4102 - loss: 1.4060 - val_accuracy: 0.3023 - val_loss: 1.7616
Epoch 8/30
77/77 ━━━━━━━━━━━━━━━━━━━━ 45s 531ms/step - accuracy: 0.4233 - loss: 1.3805 - val_accuracy: 0.326

In [14]:
model.evaluate(validation_generator)

20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 466ms/step - accuracy: 0.4016 - loss: 1.7583


[1.6775987148284912, 0.4166666567325592]

In [15]:
# Configure Git
!git config --global user.name "olid-iu"
!git config --global user.email "molid.hasanbd@gmail.com"


In [16]:
# Clone the repository
!git clone YOUR_REPOSITORY_URL


fatal: repository 'YOUR_REPOSITORY_URL' does not exist
